In [25]:
from mcap_ros2.reader import read_ros2_messages
import numpy as np
import matplotlib.pyplot as plt

"""
test_turnoff = ["RosBags//TurnOffTest0805//TurnOffTest0805_0.mcap",
                "RosBags//TurnOffTest0805V2//TurnOffTest0805V2_0.mcap",
                "RosBags//TurnOffTest0805V3//TurnOffTest0805V3_0.mcap"]

"""
"""
test_PID = ["RosBags//PIDTEST0805//PIDTEST0805_0.mcap",
            "RosBags//PIDTEST0805V2//PIDTEST0805V2_0.mcap",
            "RosBags//PIDTEST0805V3//PIDTEST0805V3_0.mcap",
            "RosBags//PIDTEST0805V4//PIDTEST0805V4_0.mcap",
            "RosBags//PIDTEST0805V5//PIDTEST0805V5_0.mcap"]

"""

test_turnoff = "RosBags\\ViconTest0605\\ViconTest0605_0.mcap"


def mcap(mcap_path):
    columns = []
    for msg in read_ros2_messages(mcap_path):
        print(msg)
        data = msg.ros_msg.data
        for i, value in enumerate(data):
            if len(columns) <= i:
                columns.append([])  # Add a new list for this index
            columns[i].append(value)
    return columns

def plot_columns(columns):
    column_labels = ['X', 'Y', 'Z', 'Roll', 'Pitch', 'Yaw']
    y_axis = ['Position[m]', 'Angle[rad]']
    for i, column in enumerate(columns):
        plt.figure()
        plt.plot(column, marker='o', linestyle='-', markersize=1, label=column_labels[i])
        plt.title(f'Plot of {column_labels[i]}')
        plt.xlabel('Time(s)')
        plt.ylabel(y_axis[i//3])
        plt.legend()
        plt.grid(True)
    plt.show()

col = mcap(test_turnoff)




FileNotFoundError: [Errno 2] No such file or directory: 'RosBags\\ViconTest0605\\ViconTest0605_0.mcap'

In [ ]:
plot_columns(turnoff_data[0][0])


turnoff_data = [mcap(test) for test in test_turnoff]
PID_data = [mcap(test) for test in test_PID]





